<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Capacitated Facility Location - Problem #A



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)
- [Lowest Production Heuristic](#lowProd)
- [Greatest Production Heuristic](#maxProd)



<br>
<br>


<a id='prob'></a>
## Problem #A

<br>
Facilities <br>
$|I| = 9$ 
<br>
<br>
Clients <br>
$|J| = 2$ 

<br>
<br>
$\alpha = 1$ (cost per mile of driving)

<br>
<br>
<br>

| Clients| Coordinates | Demand | 
| --- | --- | --- |
| Client 1 | (0,1.5) | 600 |
| Client 2 | (2.5,1.2) | 700 |

<br>
<br>

| Facilities | Coordinates | Fixed Cost | Max Production  |
| --- | --- |  --- |  --- |
| Warehouse 1 | (0,0) | 3 | 500 |
| Warehouse 2 | (0,1) | 2 | 500 |
| Warehouse 3 | (0,2) | 3 | 500 |
| Warehouse 4 | (1,0) | 1 | 500 |
| Warehouse 5 | (1,1) | 3 |  500 |
| Warehouse 6 | (1,2) | 3 | 500 |
| Warehouse 7 | (2,0) | 4 | 500 |
| Warehouse 8 | (2,1) | 3 | 500 |
| Warehouse 9 | (2,2) | 2 | 500 |


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [1]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time


##### Sets and Indices #####
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
num_facilities = len(facilities)
num_customers = len(customers)

##### Parameters #####
#cos per mile
cost_per_mile = 1
#fixed cost
setup_cost = [3,2,3,1,3,3,4,3,2]
#demand of customer
dc = [600,700]
#max production
maxp= [500,500,500,500,500,500,500,500,500]

# Euclidean distance between a facility and customer sites
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

shipping_demand={}
for k, v in shipping_cost.items():
    shipping_demand[k] = v * dc[k[0]]

start = time.time()

# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')
m.addConstrs((gp.quicksum(dc[c]*y[(c,f)] for c in range(num_customers)) <= maxp[f]*x[f] for f in range(num_facilities)), name='Capacity')

##### Objective Function #####
#m.setObjective(x.prod(setup_cost)+y.prod(shipping_demand), GRB.MINIMIZE)
m.setObjective(x.prod(setup_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()

end = time.time()
print("TIME IS: ",end - start)

Using license file /Users/danielcinalli/gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-31
Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 29 rows, 27 columns and 81 nonzeros
Model fingerprint: 0x71cb125e
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 18 rows and 0 columns
Presolve time: 0.05s
Presolved: 11 rows, 27 columns, 45 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 6.419121e+00, 12 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

In [2]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('CFLP_01_Simplex.lp')


Build a warehouse at location 2.

Build a warehouse at location 4.

Build a warehouse at location 9.

Client 1 receives 83.33 % of its demand  from Warehouse 2 .

Client 1 receives 16.67 % of its demand  from Warehouse 4 .

Client 2 receives 28.57 % of its demand  from Warehouse 4 .

Client 2 receives 71.43 % of its demand  from Warehouse 9 .

Optimal total: 6.93982428845362


In [4]:
from itertools import product
from math import sqrt
from copy import copy
import time
import random

# Randomly choose a Facility
def ChooseFac_rand(item):
    return random.randint(0, len(item)-1)


# choose a Facility with the lowest Shipping Cost
def ChooseFac_lowest_shipping(cp_shipping_cost, client, cp_facilities, facilities):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif aux[aux_key] > cp_shipping_cost.get((client, facilities.index(j))):
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))


    return aux_key



# choose a Facility with the lowest Shipping Cost
def ChooseFac_greatest_shipping(cp_shipping_cost, client, cp_facilities, facilities):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif not aux[aux_key] > cp_shipping_cost.get((client, facilities.index(j))):
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, facilities.index(j)))
            aux_key = ((client, j))


    return aux_key




# choose a Facility with the lowest Shipping Cost
def ChooseFac_lowest_shipping_fcost(cp_shipping_cost, client, cp_facilities, facilities, setup_cost):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        dedo = facilities.index(j)
        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif aux[aux_key] > cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]:
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))


    return aux_key




# choose a Facility with the lowest Shipping Cost
def ChooseFac_greatest_shipping_fcost(cp_shipping_cost, client, cp_facilities, facilities, setup_cost):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        dedo = facilities.index(j)
        # empty list for the Client
        if not aux:
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))
            # print(aux)
            # print(aux_key)
        #if this FAC is cheaper for this client, replace the item
        elif not aux[aux_key] > cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]:
            # print("....")
            # print (i[0],j[0])
            # print(aux[aux_key])
            # print(shipping_cost.get((i[0],j[0])))
            # print ("  ")
            aux.pop(aux_key)
            aux[(client, j)] = cp_shipping_cost.get((client, dedo)) + setup_cost[dedo]
            aux_key = ((client, j))


    return aux_key




# choose a Facility with the lowest Shipping Cost
def ChooseFac_lowest_production(cp_shipping_cost, client, cp_facilities, facilities, cp_maxp):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        dedo = facilities.index(j)
        if cp_maxp[dedo] != 0:
            if not aux:
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))
                # print(aux)
                # print(aux_key)
            #if this FAC is cheaper for this client, replace the item
            elif aux[aux_key] > cp_maxp[dedo]: #cp_shipping_cost.get((client, facilities.index(j))):
                # print("....")
                # print (i[0],j[0])
                # print(aux[aux_key])
                # print(shipping_cost.get((i[0],j[0])))
                # print ("  ")
                aux.pop(aux_key)
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))


    return aux_key


# choose a Facility with the lowest Shipping Cost
def ChooseFac_greatest_production(cp_shipping_cost, client, cp_facilities, facilities, cp_maxp):

    aux = {}
    aux_key = ()

    for j in cp_facilities:

        # empty list for the Client
        dedo = facilities.index(j)
        if cp_maxp[dedo] != 0:
            if not aux:
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))
                # print(aux)
                # print(aux_key)
            #if this FAC is cheaper for this client, replace the item
            elif not aux[aux_key] > cp_maxp[dedo]: #cp_shipping_cost.get((client, facilities.index(j))):
                # print("....")
                # print (i[0],j[0])
                # print(aux[aux_key])
                # print(shipping_cost.get((i[0],j[0])))
                # print ("  ")
                aux.pop(aux_key)
                aux[(client, j)] = cp_maxp[dedo] #cp_shipping_cost.get((client, facilities.index(j)))
                aux_key = ((client, j))


    return aux_key

# Calculate the Total Cost
def CalculateCost(customers, cp_facilities, cp_setup_cost, cost_per_mile, result):
    total_dist = 0
    total_setup = 0
    for client, fac, prods in result:
        #shipping cost of demand
        #total_dist += prods * cost_per_mile * compute_distance(customers[client], fac)
        total_dist += cost_per_mile * compute_distance(customers[client], fac)

        #cost of FAC setup
        if(fac in cp_facilities):
            dedo = cp_facilities.index(fac)
            total_setup += cp_setup_cost[dedo]
            #remove FAC from the list
            del cp_setup_cost[dedo]
            del cp_facilities[dedo]


    return total_dist+total_setup


def CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile, result, dc):
    total_dist = 0
    total_setup = 0
    my_facilities = copy(cp_facilities)

    for client, fac, prods in result:
        #shipping cost of demand
        #total_dist += prods * cost_per_mile * compute_distance(customers[client], fac)
        prod_perc = float(prods)/dc[client]
        total_dist += prod_perc*cost_per_mile * compute_distance(customers[client], fac)

        #cost of FAC setup
        if(fac in cp_facilities):
            dedo = cp_facilities.index(fac)
            total_setup += cp_setup_cost[dedo]
            #remove FAC from the list
            del cp_setup_cost[dedo]
            del cp_facilities[dedo]


    return total_dist+total_setup



def PrintResult(result, facs):

    #length Facs
    lenFacs = len(facs)

    #print clients and facs disposition
    for i in result:
        #print("Client " + str(i[0] + 1) + "receives part of its demand  from Warehouse" + str(i[1] + 1) + ".")
        print("Client " + str(i[0]+1) + " receives part of its demand  from Warehouse %s." % (i[1],))

        if (i[1] in facs):
            dedo= facs.index(i[1])
            del facs[dedo]

    lenFacs = lenFacs - len(facs)
    print("Number of Warehouses: " + str(lenFacs))



# Euclidean distance between a facility and customer sites
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)



<br>
<br>

<a id='random'></a>
### Random Heuristic 





In [15]:

result = []
#i = 0

start = time.time()


#copies of facilities, demands and production
#to calculate temp values
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)


# choose the Facility for each customer
for idx,j in enumerate(customers):

    while cp_dc[idx] > 0:
        fac = ChooseFac_rand(cp_facilities)

        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[fac]<=0): #facility produces more than needed
            cp_maxp[fac] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, cp_facilities[fac], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[fac]
            # append the result: Client | FAC | total of products
            result.append((idx, cp_facilities[fac], cp_maxp[fac]))
            cp_maxp[fac] = 0

            #remove facility and its production item
            del cp_facilities[fac]
            del cp_maxp[fac]


# calculate the setup_cost
cp_facilities = copy(facilities)
cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)


13.346532287098688
TIME IS:  0.0016970634460449219
Client 1 receives part of its demand  from Warehouse (0, 2).
Client 1 receives part of its demand  from Warehouse (1, 1).
Client 2 receives part of its demand  from Warehouse (1, 0).
Client 2 receives part of its demand  from Warehouse (2, 0).
Number of Warehouses: 4


<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [16]:

start = time.time()
#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx,j in enumerate(customers):

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_lowest_shipping(cp_shipping_cost, idx, cp_facilities, facilities)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)




11.154196947139797
TIME IS:  0.00522613525390625
Client 1 receives part of its demand  from Warehouse (0, 1).
Client 1 receives part of its demand  from Warehouse (0, 2).
Client 2 receives part of its demand  from Warehouse (2, 1).
Client 2 receives part of its demand  from Warehouse (2, 2).
Number of Warehouses: 4


<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [17]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx,j in enumerate(customers):

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_greatest_shipping(cp_shipping_cost, idx, cp_facilities, facilities)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)


17.15766640069072
TIME IS:  0.00780797004699707
Client 1 receives part of its demand  from Warehouse (2, 0).
Client 1 receives part of its demand  from Warehouse (2, 2).
Client 2 receives part of its demand  from Warehouse (0, 0).
Client 2 receives part of its demand  from Warehouse (0, 2).
Number of Warehouses: 4


<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [18]:
start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx,j in enumerate(customers):

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_lowest_shipping_fcost(cp_shipping_cost, idx, cp_facilities, facilities, setup_cost)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)


7.219121190746247
TIME IS:  0.003531932830810547
Client 1 receives part of its demand  from Warehouse (0, 1).
Client 1 receives part of its demand  from Warehouse (1, 0).
Client 2 receives part of its demand  from Warehouse (1, 0).
Client 2 receives part of its demand  from Warehouse (2, 2).
Number of Warehouses: 3


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [19]:


start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx,j in enumerate(customers):

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_greatest_shipping_fcost(cp_shipping_cost, idx, cp_facilities, facilities, setup_cost)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)


18.15766640069072
TIME IS:  0.01055288314819336
Client 1 receives part of its demand  from Warehouse (2, 0).
Client 1 receives part of its demand  from Warehouse (2, 1).
Client 2 receives part of its demand  from Warehouse (0, 0).
Client 2 receives part of its demand  from Warehouse (0, 2).
Number of Warehouses: 4


<a id='lowProd'></a>
### Lowest Production Heuristic



In [20]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx,j in enumerate(customers):

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_lowest_production(cp_shipping_cost, idx, cp_facilities, facilities, cp_maxp)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)


11.891417877937844
TIME IS:  0.0023508071899414062
Client 1 receives part of its demand  from Warehouse (0, 0).
Client 1 receives part of its demand  from Warehouse (0, 1).
Client 2 receives part of its demand  from Warehouse (0, 1).
Client 2 receives part of its demand  from Warehouse (0, 2).
Number of Warehouses: 3


<a id='maxProd'></a>
### Greatest Production Heuristic



In [21]:

start = time.time()

#copies of facilities, demands and production
#to calculate temp values
result = []
cp_facilities = copy(facilities)
cp_maxp= copy(maxp)
cp_dc = copy(dc)
cp_setup_cost = copy(setup_cost)
cp_shipping_cost = copy(shipping_cost)

for idx,j in enumerate(customers):

    #while still demand to be covered
    while cp_dc[idx] > 0:
        fac = ChooseFac_greatest_production(cp_shipping_cost, idx, cp_facilities, facilities, cp_maxp)
        #get index number of fac
        faci = facilities.index(fac[1])


        #update TOTAL Facility Prod and Customer demand
        if(cp_dc[idx] - cp_maxp[faci]<=0): #facility produces more than needed
            cp_maxp[faci] -= cp_dc[idx]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_dc[idx]))
            cp_dc[idx] = 0
        else:
            cp_dc[idx]  -= cp_maxp[faci]
            # append the result: Client | FAC | total of products
            result.append((idx, facilities[faci], cp_maxp[faci]))
            cp_maxp[faci] = 0

            #remove facility and its production item
            faci_cp = cp_facilities.index(fac[1])
            del cp_facilities[faci_cp]
            #del cp_maxp[faci]
            del cp_shipping_cost[(idx,faci)]

    #redo cp_facs
    cp_facilities = copy(facilities)


# calculate the setup_cost
#cp_facilities = copy(facilities)
#cp_setup_cost = copy(setup_cost)
total = CalculateCost_CFLP(customers, cp_facilities, cp_setup_cost, cost_per_mile,  result, dc)
print(total)

end = time.time()
print("TIME IS: ", end - start)

cp_facilities = copy(facilities)
PrintResult(result, cp_facilities)


15.475838527094545
TIME IS:  0.006525278091430664
Client 1 receives part of its demand  from Warehouse (2, 2).
Client 1 receives part of its demand  from Warehouse (2, 1).
Client 2 receives part of its demand  from Warehouse (2, 0).
Client 2 receives part of its demand  from Warehouse (1, 2).
Number of Warehouses: 4
